In [1]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Student_Mental_Health/Student_Mental_Health/research'

In [2]:
import os

In [3]:
os.chdir("../.")

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Student_Mental_Health/Student_Mental_Health'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluatioConfig:
    root_dir: Path
    model_path: Path
    X_test_processed: Path
    y_test_processed: Path
    save_param: Path

In [6]:
from src.constants import *
from src.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.data_root])

    def get_model_evaluation_config(self) -> ModelEvaluatioConfig:
        config = self.config.Model_evaluation
        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluatioConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            X_test_processed = config.X_test_processed,
            y_test_processed = config.y_test_processed,
            save_param = config.save_param
        )
        return model_evaluation_config


In [8]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report,roc_auc_score
import pandas as pd
import pickle

In [17]:
import pickle
import pandas as pd
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

class ModelEvaluate:
    def __init__(self,config:ModelEvaluatioConfig):
        self.config = config

    
    def metrics(self,actual,prediction):
        acc_score = accuracy_score(actual, prediction)
        conf_matrix = confusion_matrix(actual, prediction)
        class_report = classification_report(actual, prediction)
        return acc_score, conf_matrix, class_report
    
    def model_score(self):
        model = pickle.load(open(self.config.model_path, 'rb'))
        X_test = pd.read_csv(self.config.X_test_processed)
        self.y_test = pd.read_csv(self.config.y_test_processed)

        self.y_pred = model.predict(X_test)
        print(f"Model Accuracy Score: {accuracy_score(self.y_test, self.y_pred)}")
        print(f"Model Confusion Matrix: {confusion_matrix(self.y_test, self.y_pred)}")
        print(f"Model Classification Report: {classification_report(self.y_test, self.y_pred)}")

    
    def save_metrics(self):
        (acc_score, conf_matrix, class_report) = self.metrics(self.y_test, self.y_pred)
        with open(self.config.save_param, 'w') as f:
            f.write(f"acc Score: {acc_score}\n")
            f.write(f"Confusion Matrix: {conf_matrix}\n")
            f.write(f"Classification Report: {class_report}\n")

In [19]:
config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation = ModelEvaluate(config = model_evaluation_config)
model_evaluation.model_score()
model_evaluation.save_metrics()

[2024-08-29 11:44:58,054: INFO: common: Yaml file: config/config.yaml loaded successfully]
[2024-08-29 11:44:58,057: INFO: common: Yaml file: params.yaml loaded successfully]
[2024-08-29 11:44:58,059: INFO: common: created directory at: data]
[2024-08-29 11:44:58,064: INFO: common: created directory at: data/Model_evaluation]
Model Accuracy Score: 0.6111111111111112
Model Confusion Matrix: [[1 0 0 0 0]
 [1 1 1 0 0]
 [0 2 1 0 0]
 [0 1 2 4 0]
 [0 0 0 0 4]]
Model Classification Report:               precision    recall  f1-score   support

           1       0.50      1.00      0.67         1
           2       0.25      0.33      0.29         3
           3       0.25      0.33      0.29         3
           4       1.00      0.57      0.73         7
           5       1.00      1.00      1.00         4

    accuracy                           0.61        18
   macro avg       0.60      0.65      0.59        18
weighted avg       0.72      0.61      0.64        18

